# New Section

In [1]:
%matplotlib
import matplotlib.pyplot as plt

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


Using matplotlib backend: agg


In [4]:
model_data='s3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-12-11-28-10-280/output/model.tar.gz'

In [5]:
model_data

's3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-12-11-28-10-280/output/model.tar.gz'

In [6]:
from sagemaker import image_uris


In [7]:
image=image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

In [8]:
image 

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [9]:
role

'arn:aws:iam::479862333671:role/service-role/AmazonSageMaker-ExecutionRole-20220103T082598'

In [10]:
trainedmodel = sagemaker.model.Model(
    image_uri=image,
    model_data = model_data,
    role=role)

In [11]:
model_transformer = trainedmodel.transformer(instance_count=1, instance_type='ml.c5.4xlarge')

In [12]:
X_val_location = 's3://{}/{}'.format(bucket, 'data/X_val_values.csv')

In [13]:
model_transformer.transform(X_val_location, content_type='text/csv', split_type='Line')

..............................
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocation

In [14]:
model_transformer.wait()

[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:12:18:08:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
     

In [15]:
!aws s3 cp --recursive $model_transformer.output_path $data_dir

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: paths


In [2]:
data_dir = ''


In [3]:
Y_pred = pd.read_csv('s3://{}/{}'.format(bucket, 'data/X_val_values.csv.out'), header=None)
Y_pred.columns = ['demand']

In [4]:
Y_pred

,demand
0,0.471162
1,0.517790
2,0.536669
3,0.517790
4,0.471162
...,...
823225,1.077206
823226,0.504309
823227,1.342117
823228,1.404573


In [5]:
Y_test = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))['demand'].to_frame()

In [6]:
Y_test

,demand
0,0
1,0
2,0
3,2
4,0
...,...
823225,1
823226,1
823227,0
823228,1


In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mean_squared_error(Y_test, Y_pred, squared=True)


6.131172588601535

In [24]:
mean_squared_error(Y_test, Y_pred, squared=False)

2.476120471342526

In [26]:
mean_absolute_error(Y_test, Y_pred)

1.2007927503676232